In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

/home/vipuser/anaconda3/envs/GLM4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = '/home/LLM_para/para_glm4'
# lora_path = './final_checkpoint_newS_SFT_SQL_CN_TV'
lora_path = './final_checkpoint_dataset1_SQL'
# lora_path = './final_checkpoint_newS_SFT_SQL_CN_TV'
val_dataset_path = "./mydataset2_SQL/T2Q_GLM4_SFT_val_sql.jsonl"

In [3]:
model = AutoPeftModelForCausalLM.from_pretrained(
        lora_path, trust_remote_code= True, device_map='auto')

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.52it/s]


In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
test_df = pd.read_json(val_dataset_path, lines=True)
print(test_df)

                                           instruction  \
0     I want you to act as a SQL terminal in front ...   
1     I want you to act as a SQL terminal in front ...   
2     I want you to act as a SQL terminal in front ...   
3     I want you to act as a SQL terminal in front ...   
4     I want you to act as a SQL terminal in front ...   
..                                                 ...   
173   I want you to act as a SQL terminal in front ...   
174   I want you to act as a SQL terminal in front ...   
175   I want you to act as a SQL terminal in front ...   
176   I want you to act as a SQL terminal in front ...   
177   I want you to act as a SQL terminal in front ...   

                                                 input  \
0    Find number of products which Sony does not make.   
1       How many products have prices of at least 180?   
2    Find the name, headquarter and revenue of all ...   
3             How many companies were created by Andy?   
4    Find the

In [6]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
#     print("reponse1++++++++++++++++",response)
     
    return response

In [7]:
import re
pattern = r"-- SQL: (.*?);"
test_df_check = test_df[:1]
print(test_df_check)
test_text_list = []
for index, row in test_df_check.iterrows():
    instruction = row['instruction']
    input_value = row['input']
    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"}
    ]

    response = predict(messages, model, tokenizer)
    match = re.search(pattern, response, re.DOTALL)
    if match:  
        response = match.group(1).strip() 
    else:
        sql_statement = "None"
    print("预测SQL：",response)
#     if ";" in response:
#         response = response.split(";")[0]
#     if "```SQL:" in response:
# #         print(23)
#         response = response.split("```SQL:")[1]
#     response = re.sub(r'\s+', ' ', response).strip()
# #     print(response)
    

                                         instruction  \
0   I want you to act as a SQL terminal in front ...   

                                               input  \
0  Find number of products which Sony does not make.   

                                              output  
0  ```gerenated SQL\n-- SQL: SELECT COUNT(DISTINC...  
预测SQL： SELECT COUNT(DISTINCT name) FROM products WHERE name NOT IN (SELECT t1.name FROM products as t1 JOIN manufacturers as t2 ON t1.manufacturer  =  t2.code WHERE t2.name  =  'Sony')


In [8]:
#  read glod 
from tqdm import tqdm
# 录入db_id query用于glod  第一次使用需要
# df = pd.read_csv("./mydataset_SQL/validation_dataset_formatted.csv",encoding="GBK")

df = pd.read_csv("./mydataset2_SQL/val_Myidea_tableOnly.csv",encoding="utf-8")
results = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    results.append([row['query'],row['question'],row['db_id']])
new_df = pd.DataFrame(results, columns = ['reference_query','question','db_id'])
print(new_df)

100%|██████████████████████████████████████████████████████████████████████████████| 178/178 [00:00<00:00, 18331.03it/s]

                                       reference_query  \
0    SELECT COUNT(DISTINCT name) FROM products WHER...   
1     SELECT COUNT(*) FROM products WHERE price >= 180   
2    SELECT name ,  headquarter ,  revenue FROM man...   
3    SELECT COUNT(*) FROM manufacturers WHERE found...   
4    SELECT name FROM manufacturers WHERE revenue B...   
..                                                 ...   
173  SELECT name ,  built_year FROM mill WHERE type...   
174  SELECT builder ,  COUNT(*) FROM railway GROUP ...   
175             SELECT wheels ,  location FROM railway   
176                   SELECT title ,  issues FROM book   
177  SELECT t1.title ,  t2.publication_date FROM bo...   

                                              question          db_id  
0    Find number of products which Sony does not make.  manufactory_1  
1       How many products have prices of at least 180?  manufactory_1  
2    Find the name, headquarter and revenue of all ...  manufactory_1  
3             H

In [9]:
import re
import time
start_time = time.time()
index = 0
pattern = r"-- SQL: (.*?);"
predict_list = []
for index, row in test_df.iterrows():
    index += 1
    instruction = row['instruction']
    input_value = row['input']
    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"}
    ] 
    response = predict(messages, model, tokenizer)
    response = predict(messages, model, tokenizer)
    match = re.search(pattern, response, re.DOTALL)
    if match:  
        response = match.group(1).strip() 
    else:
        sql_statement = "None"
    print(f"预测SQL{index}：",response)
    predict_list.append(response)
end_time = time.time()
#     messages.append({"role": "assistant", "content": f"{response}"})
#     result_text = f"{messages[0]}\n\n{messages[1]}\n\n{messages[2]}"
#     print(result_text)
    
#     test_text_list.append(swanlab.Text(result_text, caption=response))

预测SQL1： SELECT COUNT(DISTINCT name) FROM products WHERE name NOT IN (SELECT t1.name FROM products as t1 JOIN manufacturers as t2 ON t1.manufacturer  =  t2.code WHERE t2.name  =  'Sony')
预测SQL2： SELECT COUNT(*) FROM products WHERE price  >=  180
预测SQL3： SELECT name ,  headquarter ,  revenue FROM manufacturers ORDER BY revenue DESC
预测SQL4： SELECT COUNT(*) FROM manufacturers WHERE founder  =  'Andy'
预测SQL5： SELECT name FROM manufacturers WHERE revenue BETWEEN 100 AND 150
预测SQL6： SELECT t1.name FROM manufacturers as t1 JOIN products as t2 ON t1.code  =  t2.manufacturer GROUP BY t1.name HAVING AVG(t2.price) >= 150
预测SQL7： SELECT name FROM manufacturers WHERE revenue  >  (SELECT AVG(revenue) FROM manufacturers)
预测SQL8： SELECT name FROM products WHERE price  <=  200
预测SQL9： SELECT t2.fname ,  t2.age FROM sportsinfo as t1 JOIN student as t2 ON t1.stuid  =  t2.stuid WHERE t1.sportname  =  "Football" INTERSECT SELECT t2.fname ,  t2.age FROM sportsinfo as t1 JOIN student as t2 ON t1.stuid  =  t2.

预测SQL68： SELECT date FROM game ORDER BY home_team DESC
预测SQL69： SELECT t1.name FROM physician as t1 JOIN patient as t2 ON t1.employeeid  =  t2.pcp GROUP BY t1.employeeid HAVING COUNT(*)  >  1
预测SQL70： SELECT department FROM employees GROUP BY department ORDER BY COUNT(department) DESC LIMIT 1
预测SQL71： SELECT name ,  cost FROM procedures ORDER BY cost DESC
预测SQL72： SELECT t3.name FROM procedures as t3 JOIN trained_in as t1 ON t3.code  =  t1.treatment JOIN physician as t2 ON t1.physician  =  t2.employeeid WHERE t2.name  =  "John Wen" AND t3.cost  <  5000
预测SQL73： SELECT t1.name FROM physician as t1 JOIN prescribes as t2 ON t1.employeeid = t2.physician ORDER BY t2.dose DESC LIMIT 1
预测SQL74： SELECT appointmentid FROM appointment ORDER BY start DESC LIMIT 1
预测SQL75： SELECT DISTINCT t1.name FROM nurse as t1 JOIN on_call as t2 ON t1.employeeid  =  t2.nurse
预测SQL76： SELECT name FROM procedures WHERE cost  >  1000 EXCEPT SELECT t3.name FROM physician as t1 JOIN trained_in as t2 ON t1.employeeid

预测SQL120： SELECT campus FROM campuses WHERE location  =  "Chico"
预测SQL121： SELECT COUNT(*) FROM campuses WHERE county  =  "Los Angeles"
预测SQL122： SELECT COUNT(*) FROM campuses WHERE county  =  "LA"
预测SQL123： SELECT campus FROM campuses WHERE county  =  "Los Angeles"
预测SQL124： SELECT degrees FROM campuses as t1 JOIN degrees as t2 ON t1.id  =  t2.campus WHERE t1.campus  =  "San Jose State University" AND t2.year  =  2000
预测SQL125： SELECT t1.faculty FROM faculty as t1 JOIN campuses as t2 ON t1.campus  =  t2.id WHERE t2.campus  =  "Long Beach State University" AND t1.year  =  2002
预测SQL126： SELECT COUNT(*) FROM campuses
预测SQL127： SELECT nationality FROM journalist GROUP BY nationality ORDER BY COUNT(*) DESC LIMIT 1
预测SQL128： SELECT t3.title FROM authors as t1 JOIN authorship as t2 ON t1.authid  =  t2.authid JOIN papers as t3 ON t2.paperid  =  t3.paperid WHERE  t1.fname  =  "Olin" AND t1.lname  =  "Shivers"
预测SQL129： SELECT COUNT(DISTINCT t3.paperid) FROM inst as t1 JOIN authorship as t2 ON

In [10]:
total_data = len(df['question'])
total_time = end_time - start_time
# print(total_data)
signal_infernece_time = total_time/total_data
print(signal_infernece_time)

4.244172764628121


In [11]:
new_df['generated_query'] = predict_list
new_df.to_csv("generated_SQL_DTSSQL2.csv", index=False)

In [12]:
def append_string_to_file(text, file_path):
    with open(file_path, 'a') as file:
        file.write(text + '\n')

def remove_spaces(text):
    return re.sub(r'\s+', ' ', text)

In [13]:
for index, row in new_df.iterrows():
    print(f"Processing the {index}th rows")
    if pd.isna(row['generated_query']):
        print(row['generated_query'])
        sql_query = input("give me the correct SQL query")
        sql_query = remove_spaces(sql_query)
        append_string_to_file(sql_query, "Predicted_SQL2.txt")
        append_string_to_file(row['reference_query'] + "\t" + row['db_id'], "Gold.txt")
    elif row['generated_query'][:6] == "SELECT":
        append_string_to_file(remove_spaces(row['generated_query']), "Predicted_SQL2.txt")
        append_string_to_file(row['reference_query'] + "\t" + row['db_id'], "Gold.txt")
    else:
        print(row['generated_query'])
        sql_query = input("give me the correct SQL query")
        sql_query = remove_spaces(sql_query)
        append_string_to_file(sql_query, "Predicted_SQL2.txt")
        append_string_to_file(row['reference_query'] + "\t" + row['db_id'], "Gold.txt")

Processing the 0th rows
Processing the 1th rows
Processing the 2th rows
Processing the 3th rows
Processing the 4th rows
Processing the 5th rows
Processing the 6th rows
Processing the 7th rows
Processing the 8th rows
Processing the 9th rows
Processing the 10th rows
Processing the 11th rows
Processing the 12th rows
Processing the 13th rows
Processing the 14th rows
Processing the 15th rows
Processing the 16th rows
Processing the 17th rows
Processing the 18th rows
Processing the 19th rows
Processing the 20th rows
Processing the 21th rows
Processing the 22th rows
Processing the 23th rows
Processing the 24th rows
Processing the 25th rows
Processing the 26th rows
Processing the 27th rows
Processing the 28th rows
Processing the 29th rows
Processing the 30th rows
Processing the 31th rows
Processing the 32th rows
Processing the 33th rows
Processing the 34th rows
Processing the 35th rows
Processing the 36th rows
Processing the 37th rows
Processing the 38th rows
Processing the 39th rows
Processing